In [1]:
!which python

/Users/yangyang/miniconda2/bin/python


In [2]:
!pip install pattern

    100% |████████████████████████████████| 24.6MB 27kB/s eta 0:00:01
  Running setup.py bdist_wheel for pattern ... - \ | / - done
  Stored in directory: /Users/yangyang/Library/Caches/pip/wheels/88/c3/56/c85103e2876796af4a892df2367a879007d36d74511bf47d42
Successfully built pattern


In [2]:
!pip install nltk

  Using cached nltk-3.2.2.tar.gz
  Running setup.py bdist_wheel for nltk ... - \ | done
  Stored in directory: /Users/yangyang/Library/Caches/pip/wheels/42/b5/27/718985cd9719e8a44a405d264d98214c7a607fb65f3a006f28
Successfully built nltk


In [9]:
!ls

Untitled.ipynb    X_features_train  contractions.py   normalization.pyc
Untitled1.ipynb   Y_labels_test     contractions.pyc
X_features_test   Y_labels_train    normalization.py


In [ ]:
type(g_mox)

In [13]:
import pandas as pd
import numpy as np
import sys
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
import gensim
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import normalization


def get_topn_tags_transform(path,topn,tag):
    """
    read cleaned data and transform them into one tag per row
    1. get top n tags
    2. expand row
    3. to boolean
    4. aggregate by content
    """
    df = pd.read_csv(path,quotechar='|',sep=',',header=None)
    df.columns = ['title','body','tags']
    merged = [ title + ' ' + body for title, body in zip(df.title,df.body)]
    df_merged = pd.DataFrame({'content':merged,'tags':df.tags.copy()})
    df_merged.tags = df_merged.tags.apply(lambda x: x.replace('<','').split('>')[:-1])
    df_transformed = pd.DataFrame(df_merged.tags.tolist(),index=df_merged.content).stack().reset_index()[['content',0]]
    df_transformed.columns = ['content','tags']
    top_tags = Counter(df_transformed.tags).most_common()[:topn]
    top_n_tags = [tag for tag, num in top_tags]
    df_filtered = df_transformed[df_transformed.tags.apply(lambda x: x in set(top_n_tags))]
    df_filtered.tags = [int(bool) for bool in df_filtered.tags == tag]
    df_filtered.columns = ['content','is_{}'.format(tag)]
    rslt = df_filtered.groupby('content')['is_{}'.format(tag)].agg(['sum']).reset_index()
    rslt.columns = ['content','is_{}'.format(tag)]
    return rslt, top_n_tags




class ExtractAverageWordVectors(BaseEstimator,TransformerMixin):
    def __init__(self,g_model):
        self.g_model = g_model

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        if self.g_model:
            return self.averaged_word_vectors(X) 

    def averaged_word_vectors(self, tokenized_list):
        weighted_ave = []

    
        for sentence in tqdm(tokenized_list):
            word_vecs =  np.array([self.g_model[word] for word in sentence if word in self.g_model])
        
            weighted_ave.append(np.sum(word_vecs, axis = 0)/len(word_vecs))
    
        return np.array(weighted_ave)



class ExtractTfidfAveVec(BaseEstimator,TransformerMixin):
    def __init__(self,corpus, g_model):
        self.corpus = corpus
        self.g_model = g_model

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return self.ave_weighted_tfidf(X) 

    def tfidf_extractor(self, ngram_range=(1,1)):

        tfidf_obj = TfidfVectorizer(min_df=3, strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}', ngram_range=ngram_range, use_idf = 1, smooth_idf = 1, sublinear_tf = 1, stop_words='english')

        tfidf_features = tfidf_obj.fit_transform(self.corpus)
        
        return tfidf_obj, tfidf_features

    def tfidf_mapper(self, tfidf_obj, tfidf_features):
        vocab = tfidf_obj.vocabulary_
        words = vocab.keys()
        word_tfidfs = [tfidf_features[0, vocab.get(word)] if vocab.get(word) else 0 for word in words]
        word_tfidf_map = {word:tfidf_val for word, tfidf_val in zip(words, word_tfidfs)}
        
        return word_tfidf_map

    def ave_weighted_tfidf(self, tokenized_list):
        self.tokenized_list = tokenized_list
        weighted_ave = []
        tfidf_obj, tfidf_features = self.tfidf_extractor()
        word_tfidf_map = self.tfidf_mapper(tfidf_obj, tfidf_features ) 

        for sentence in tqdm(self.tokenized_list):
            word_vecs =  np.array([self.g_model[word] * self.word_in_word_tfidf_map(word, word_tfidf_map)  for word in sentence if word in self.g_model])

            weighted_ave.append(np.sum(word_vecs, axis = 0)/len(word_vecs))

        return np.array(weighted_ave)
    
    def word_in_word_tfidf_map(self, word, word_tfidf_map):
        if word in word_tfidf_map.values():
            return word_tfidf_map[word]
        else: 
            return 1

class Doc2Vec(BaseEstimator,TransformerMixin):
    def __init__():
        pass
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        pass








In [37]:
def token_strings(X):
    t = list(X[:].values.flatten())

    return normalization.normalize_corpus(t, True)


In [38]:
print("="*32)
print("STACKOVERFLOW QUESTION TAGGER")
print("="*32)

# question = input("Please enter the stackoverflow question to be tagged:\n ")

# print(question)
df, _ = get_topn_tags_transform("../data/processed/stack_ds_4_9_2017 .csv",10,'java')


X_train, X_test, y_train, y_test = train_test_split(df.ix[:,:-1], df.ix[:,-1], test_size=0.3, random_state=123)


model_word_two_vec = gensim.models.Word2Vec(token_strings(X_train), size=10, window=10, min_count=2, sample=1e-3)

eawv = ExtractAverageWordVectors(model_word_two_vec)
# wv_tfidf = ExtractTfidfAveVec(corpus=t, g_model=model_word_two_vec)

x_train = eawv.transform(token_strings(X_train))



# model_word_two_vec = gensim.models.Word2Vec(token_strings(X_test), size=10, window=10, min_count=2, sample=1e-3)

# eawv = ExtractAverageWordVectors(model_word_two_vec)



x_test = eawv.transform(token_strings(X_test))



print(len(X))
print(X[:4])

STACKOVERFLOW QUESTION TAGGER


100%|██████████| 715/715 [00:00<00:00, 4424.64it/s]

715
[[ 1.99864554 -0.47518596 -2.09556365 -0.55850607 -0.42836574 -1.5323267
   1.26716053  0.28056291 -0.06970328 -0.7226606 ]
 [ 2.50238204 -0.54159051 -2.56221032 -0.61460739 -0.46356079 -1.81360543
   1.53762889  0.315047   -0.21601747 -0.77697396]
 [ 2.62299204 -0.57335645 -2.75298929 -0.76352215 -0.54372853 -2.02788305
   1.59860218  0.34102121 -0.20574662 -0.85044068]
 [ 2.57184863 -0.41191912 -2.49855614 -0.63864875 -0.4543162  -1.8745383
   1.41518617  0.32180277 -0.38194978 -0.4583368 ]]


In [28]:
t = list(X_test[:].values.flatten())

tokenized_text = normalization.normalize_corpus(t, True)


# wv_tfidf = ExtractTfidfAveVec(corpus=t, g_model=model_word_two_vec)

X = eawv.transform(tokenized_text)
# y_test

100%|██████████| 715/715 [00:00<00:00, 4503.77it/s]


In [20]:
from glob2 import glob
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator,TransformerMixin
from gensim import models,corpora
import numpy as np
import time
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score,RandomizedSearchCV
import xgboost as xgb
from sklearn.metrics import recall_score
import pickle
import re


In [96]:
from sklearn import svm

In [106]:
model = svm.SVC(kernel='linear',class_weight={1:15})

In [107]:
model.fit(x_train, y_train)

SVC(C=1.0, cache_size=200, class_weight={1: 15}, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [108]:
y_pred = model.predict(x_test)

In [116]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0,

In [109]:
recall_score(y_pred=y_pred, y_true=y_test)

0.6428571428571429

In [110]:
accuracy_score(y_pred=y_pred,y_true=y_test)

0.86573426573426571

In [102]:
def my_train_test_split(X,Y):
#     X = df.iloc[:, :-1].values
#     Y = df.iloc[:, -1].values
    skf = StratifiedShuffleSplit(n_splits=1, random_state=123)
    x_train, y_train, x_test, y_test = None,None,None,None,
    for train_i, test_i in skf.split(X, Y):
        x_train, y_train = X[train_i], Y[train_i]
        x_test, y_test = X[test_i], Y[test_i]
    return x_train,y_train,x_test,y_test


def random_search(x_train,y_train,num_iter=10):
    param_distribs = {
        'max_depth': [3, 4],
        'learning_rate': [0.01, 0.05, 0.1, 0.5],
        'n_estimators': list(range(100,1000,100)),
        'colsample_bytree': [0.5,0.7,0.9],
    }

    gbm = xgb.XGBClassifier()
    rnd_search = RandomizedSearchCV(gbm, param_distribs, n_iter=num_iter, cv=5, scoring='accuracy')
    rnd_search.fit(x_train, y_train)

    cvres = rnd_search.cv_results_
    scores_list = []
    for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
        scores_list.append((mean_score, params))
    highest_score, highest_paras = sorted(scores_list, key=lambda x: x[0], reverse=True)[0]
    print('best score: {}, best paras:{}'.format(highest_score, highest_paras))
    return highest_score,highest_paras



In [118]:
df.title[:]

0       How is viewport size typically passed as an ar...
1            mysqli_query insert success and throws error
2                         Android Studio - Resources Menu
3                 Creating new rows for listed substrings
4          Error when require twapi package in tcl script
5                            BufferedStrategie on JPanel?
6                                Hosting Access on Server
7       ReactJS + React Router: How to disable link fr...
8       How to summarize the elements in a complex obj...
9       How many navigationType when you use component...
10      open 5 instances of chrome with different urls...
11      Replace text in column if matches specific pat...
12            knockout arrayfirst not working as expected
13      Custom font not working with Calligraphy libra...
14      Moving to next activity after successful faceb...
15      set image size to fill frame, regardless which...
16      Table per class configuration with Hibernate O...
17            

In [90]:
_, highest_paras =random_search(x_train,y_train,num_iter=20)

best score: 0.939412117576, best paras:{'n_estimators': 100, 'learning_rate': 0.05, 'colsample_bytree': 0.7, 'max_depth': 3}


In [27]:
_, highest_paras =random_search(X,y_train,num_iter=20)

best score: 0.30557784547, best paras:{'n_estimators': 700, 'learning_rate': 0.1, 'colsample_bytree': 0.7, 'max_depth': 4}


In [ ]:
X,y_test


In [105]:
1/np.mean(y_train)

15.435185185185187

In [89]:
len(x_test),len(y_test),len(x_train),len(y_train)

(715, 715, 1667, 1667)

In [94]:
from sklearn.metrics import accuracy_score


In [95]:
gbm = xgb.XGBClassifier(
    max_depth=highest_paras['max_depth'],
    n_estimators=highest_paras['n_estimators'],
    learning_rate=highest_paras['learning_rate'],
    colsample_bytree=highest_paras['colsample_bytree']
)

gbm_model = gbm.fit(x_train,y_train)

# pickle.dump(gbm_model,'./data/xgb_{}.pkl'.format(tag_name))
preds = gbm_model.predict(x_test)
print('final model test on test set. Score:{}'.format(accuracy_score(y_test,preds)))

final model test on test set. Score:0.944055944056


# preprcoess

In [41]:
df = pd.read_csv('../data/processed/stack_ds_4_9_2017 .csv',quotechar='|',sep=',',header=None)

In [46]:
df.columns = ['title', 'body', 'tags']
merged = [title + ' ' + body for title, body in zip(df.title, df.body)]
df_merged = pd.DataFrame({'content': merged, 'tags': df.tags.copy()})
data_tags = df_merged.tags.apply(lambda x: x.replace('<','').split('>')[:-1])

In [49]:
topn_tags = ['javascript','java','android','php','python','c#','html','jquery','ios','css']

In [58]:
new_targets = []
for sample_tags in data_tags:
    sample_tags_wanted = []
    for tag in sample_tags:
        if tag in topn_tags:
            sample_tags_wanted.append(tag)
    new_targets.append(sample_tags_wanted)

In [64]:
bool_index = [ True if t != [] else False for t in new_targets ]

In [71]:
rslt = pd.DataFrame({'content':df_merged.content,'tags':new_targets}).loc[bool_index,:]

In [76]:
from sklearn.preprocessing import MultiLabelBinarizer

In [115]:
mlb = MultiLabelBinarizer().fit(rslt.tags)

In [ ]:
mlb.transform

In [80]:
df.shape

(4533, 3)

In [83]:
rslt.reset_index(drop=True)

,content,tags
0,How is viewport size typically passed as an ar...,[html]
1,mysqli_query insert success and throws error T...,[php]
2,BufferedStrategie on JPanel? I am programming ...,[java]
3,ReactJS + React Router: How to disable link fr...,"[javascript, html, css]"
4,open 5 instances of chrome with different urls...,[javascript]
5,Custom font not working with Calligraphy libra...,[android]
6,Moving to next activity after successful faceb...,[android]
7,"set image size to fill frame, regardless which...",[java]
8,Table per class configuration with Hibernate O...,[java]
9,Xamarin Updating and NuGet Cycles I'm starting...,[c#]


,title,body,tags
0,How is viewport size typically passed as an ar...,I'm looking at the docs for a command line too...,<html><web><command-line>
1,mysqli_query insert success and throws error,This issue has baffled me. I have a simple mys...,<php><mysqli>
2,Android Studio - Resources Menu,I am new to android Studio. I have placed a te...,<android-studio><android-resources>
3,Creating new rows for listed substrings,"My goal is to create a wordcloud in R, but I'm...",<r>
4,Error when require twapi package in tcl script,I have a script which is run from within an ap...,<tcl><packages>
5,BufferedStrategie on JPanel?,I am programming a game and at the moment I us...,<java><swing><canvas><jpanel>
6,Hosting Access on Server,What is the draw back of hosting the front end...,<database><deployment><ms-access-2010><access>
7,ReactJS + React Router: How to disable link fr...,"Using React-Router's Link, I have a set up lik...",<javascript><html><css><reactjs><react-router>
8,How to summarize the elements in a complex obj...,I have a very complex S4 object (output from a...,<r>
9,How many navigationType when you use component...,"When I am using WebView and poll other uri, I ...",<webview><react-native>
